In [73]:
import requests
html = requests.get('https://afisha.tut.by/day/film/2020-11-01/2020-11-28/')

In [74]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html.content, 'html.parser')

In [75]:
import re

def get_film_data(filmUrl):
    html = requests.get(filmUrl)
    soup = BeautifulSoup(html.content, 'html.parser')
    
    description = soup.find(id='event-description')
    if description:
        description = description.contents[2].strip()
    
    genres = soup.find('td', class_='genre')
    if genres:
        genres = genres.select('a', class_='genre');
        if genres:
            genres = list(map(lambda x: x.text, genres))
    
    year = soup.find('td', class_='year')
    if year:
        year=year.text
    
    country = soup.find('td', class_='author')
    if country:
        country = country.text.strip()
    
    duration = soup.find('td', class_='duration')
    if duration:
        duration_re = re.compile('([0-9]+)')
        duration = re.search(duration_re, duration.text.strip()).group(1)
    
    producer_re = re.compile('Режиссер: (.*)')
    producer = soup.find(text=producer_re) 
    if producer:
        producer = re.search(producer_re, producer).group(1)

    actors_re = re.compile('В ролях: (.*)')
    actors = soup.find_all('p')
    actors = [p for p in actors if p.find(text=actors_re)]; # find if there a valid p
    if len(actors):
        actors = actors[0].text
        actors_str = re.search(actors_re, actors).group(1)
        actors = list(map(lambda x: x.strip(), actors_str.split(',')))
    
    rating = soup.find(class_='rating-big__value')
    if rating:
        rating = rating.text
    
    return {
        'description': description,
        'genres': genres,
        'year': year,
        'country': country,
        'duration': duration,
        'producer': producer,
        'actors': actors,
        'rating': rating,
    }

In [76]:
from pipetools import pipe

flatten = lambda list_: [item for sublist in list_ for item in sublist]
get_li_list = (pipe 
               | (map, lambda x: x.find_all('li', class_='lists__li'))
               | list 
               | flatten
              );

In [77]:
from IPython.display import clear_output

films = []
ul_list = soup.find_all('ul', class_='list_afisha')
li_list = get_li_list(ul_list)
    

for index, film in enumerate(li_list):
    clear_output(wait=True)
    print('Processing ' + str(index + 1) + ' of ' + str(len(li_list)))

    a_name = film.find('a', class_='name')
    img = film.find('img')

    if not a_name:
        continue
    
    link_to_film = a_name['href']

    film_data = get_film_data(link_to_film)
    
    result = {
        'name': a_name.text.strip(),
        'img_url': img['src'],
        'url': link_to_film,
    }
    result.update(film_data)

    films.append(result)

Processing 100 of 100


In [78]:
import pandas as pd

df_films = pd.DataFrame(films)

In [79]:
df_films

,name,img_url,url,description,genres,year,country,duration,producer,actors,rating
0,Кольская сверхглубокая,https://img.afisha.tut.by/static/media/176x0ec...,https://afisha.tut.by/film/kolskaya-sverkhglub...,Кольская сверхглубокая скважина - самый масшта...,[Триллер],2020,Россия,None,Арсений Сюхин,"[Милена Радулович, Никита Дювбанов, Кирилл Ков...","4,0"
1,Подольские курсанты,https://img.afisha.tut.by/static/media/176x0ec...,https://afisha.tut.by/film/podolskie-kursanty/,Фильм о подвиге подольских курсантов в октябре...,"[Военный, Драма, Исторический]",2019,Россия,132,Вадим Шмелев,"[Алексей Бардуков, Евгений Дятлов, Сергей Безр...","10,0"
2,Кот под прикрытием,https://img.afisha.tut.by/static/media/176x0ec...,https://afisha.tut.by/film/kot_pod_prikrytiem/,,[Приключения],2019,None,90,None,[],"10,0"
3,Побочный эффект,https://img.afisha.tut.by/static/media/176x0ec...,https://afisha.tut.by/film/pobochnyy_effekt_94...,Однажды в размеренную и счастливую жизнь архит...,"[Триллер, Ужасы]",2020,None,100,None,[],None
4,Друзья на свою голову,https://img.afisha.tut.by/static/media/176x0ec...,https://afisha.tut.by/film/druzya_na_svoyu_gol...,"Леа и Марк, Карин и Фрэнсис – две пары закадыч...",[Комедия],2020,None,110,None,[],None
...,...,...,...,...,...,...,...,...,...,...,...
95,Ночь в Лувре: Леонардо да Винчи,https://img.afisha.tut.by/static/media/176x0ec...,https://afisha.tut.by/film/noch_v_luvre_leonar...,Эксклюзивная ночная экскурсия по выставке «Лео...,[Документальный],2020,None,100,None,[],None
96,TheatreHD: Золотая маска: Иранская конференция,https://img.afisha.tut.by/static/media/176x0ec...,https://afisha.tut.by/film/theatrehd_zolotaya_...,Эта постановка по пьесе знаменитого драматурга...,None,2020,None,120,None,[],None
97,TheatreHD: Настоящая комедия (RU SUB),https://img.afisha.tut.by/static/media/176x0ec...,https://afisha.tut.by/film/theatrehd_nastoyash...,Язык: английский,None,2019,Великобритания,160,None,[],"7,5"
98,Нулевой канал (RU SUB),https://img.afisha.tut.by/static/media/176x0ec...,https://afisha.tut.by/film/nulevoy_kanal_zero_...,Фильм демонстрируется на языке оригинала с рус...,"[Приключения, Спорт]",2020,None,130,None,[],None
